In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [ ]:
#Download the files for training
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /content/horse-or-human.zip

--2020-06-19 06:07:19--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c18::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/content/horse-or-human.zip’

/content/horse-or-h 100%[===================>] 142.65M   130MB/s    in 1.1s    

2020-06-19 06:07:21 (130 MB/s) - ‘/content/horse-or-human.zip’ saved [149574867/149574867]



In [ ]:
#Download files for validation
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /content/validation-horse-or-human.zip

--2020-06-19 06:07:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c0e::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/content/validation-horse-or-human.zip’

/content/validation 100%[===================>]  10.95M  --.-KB/s    in 0.1s    

2020-06-19 06:07:22 (78.8 MB/s) - ‘/content/validation-horse-or-human.zip’ saved [11480187/11480187]



In [ ]:
import os
import zipfile

#Unzip the training and validation test sets
local_zip = '/content/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/horse-or-human')
local_zip = '/content/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/validation-horse-or-human')
zip_ref.close()

In [ ]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/content/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/content/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/content/validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/content/validation-horse-or-human/humans')

In [ ]:
print('total training horse images:', len(os.listdir(train_horse_dir)))
print('total training human images:', len(os.listdir(train_human_dir)))
print('total validation horse images:', len(os.listdir(validation_horse_dir)))
print('total validation human images:', len(os.listdir(validation_human_dir)))

total training horse images: 500
total training human images: 527
total validation horse images: 128
total validation human images: 128


In [ ]:
#Define the model
model = tf.keras.models.Sequential([
    # The first convolution
    Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    # The second convolution
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # The third convolution
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # The fourth convolution
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # The fifth convolution
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    Flatten(),
    # 512 neuron hidden layer
    Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    Dense(1, activation='sigmoid')   #Signmoid activation because its binary classification
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),   #You import RMSprop so you can set learning rate
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255  (Normalize)
# Data augmentation occurs here too. Training data is augmented and the augmented images are used for training. The original training images ARE NOT USED for training (only the augmented ones are!)
# Only the augmented images are returned. The original is not!
# Experiment with the data augmentation parameters. Sometimes, its better without data augmentation to start with (If the test dataset is like the train dataset, of which the train dataset has it's images augmented so the originals (and hence the test dataset) are not trained upon and not recognizable by the model)

train_datagen = ImageDataGenerator(rescale=1/255,
                                        rotation_range = 40,   #Range of degree to rotate the image by
                                        width_shift_range=0.2,  #How much ratio to shift image left and right
                                        height_shift_range=0.2, #How much ratio to shift image up and down
                                        shear_range=0.2,    #How much to flatten
                                        zoom_range=0.2,    #How much to zoom in
                                        horizontal_flip=True,   #Flip the image about the y axis
                                        fill_mode='nearest')    #Attempt to recreate any missing pixels in the img

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator. The training labels will be auto created for you (Based on the name of the subdirectory 'horse' and 'human')
train_generator = train_datagen.flow_from_directory(
        '/content/horse-or-human/',  # This is the source directory for training images. Inside this folder contains 2 subdirectories (1 for horse and 1 for human).
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


validation_generator = validation_datagen.flow_from_directory(
        '/content/validation-horse-or-human/',
        target_size=(300, 300),
        batch_size=32,   #1 epoch = 1 entire pass on the data set = batch_size * steps_per_epoch.    1 step_per_epoch = 1 batch_size amount of pictures processed
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
class mycallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('accuracy')>0.98):
      print('\n 98% Accuracy reached!')
      self.model.stop_training = True

In [ ]:
callbacks = mycallback()

model.fit(
      train_generator,    #Specify the training generator defined previously. No need for labels as they are created in the train_generator
      steps_per_epoch=8,  #Since you have 1027 images, 128*8 = 1024 hence you need 8 steps of batch_size=128 to get 1024 images (And hence count it as 1 epoch)
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8,  #Since you have 256 images, 32*8 = 256 hence you need 8 steps of batch_size=32 to pass through all 256 images (and hence count it as one epoch)
      callbacks=[callbacks])  

Epoch 1/15
8/8 [==============================] - 14s 2s/step - loss: 0.7869 - accuracy: 0.5250 - val_loss: 0.6641 - val_accuracy: 0.5820
Epoch 2/15
8/8 [==============================] - 14s 2s/step - loss: 0.8178 - accuracy: 0.6096 - val_loss: 0.6739 - val_accuracy: 0.5781
Epoch 3/15
8/8 [==============================] - 13s 2s/step - loss: 0.5327 - accuracy: 0.8242 - val_loss: 3.3691 - val_accuracy: 0.5078
Epoch 4/15
8/8 [==============================] - 13s 2s/step - loss: 0.3158 - accuracy: 0.8732 - val_loss: 1.2874 - val_accuracy: 0.7031
Epoch 5/15
8/8 [==============================] - 13s 2s/step - loss: 0.3076 - accuracy: 0.8732 - val_loss: 1.3172 - val_accuracy: 0.6289
Epoch 6/15
8/8 [==============================] - 13s 2s/step - loss: 0.1562 - accuracy: 0.9355 - val_loss: 1.5993 - val_accuracy: 0.6680
Epoch 7/15
8/8 [==============================] - 13s 2s/step - loss: 0.2058 - accuracy: 0.9209 - val_loss: 1.8418 - val_accuracy: 0.6016
Epoch 8/15
8/8 [==================

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving human01-17.png to human01-17.png
[1.]
human01-17.png is a human
